In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload
import lib2
from IPython.display import clear_output
from lib import data_management as dm
from time import sleep
import drivers
from lib2.IQPulseSequence import *
from drivers.KeysightAWG import *

warnings.filterwarnings("ignore",".*GUI is implemented.*")
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

In [3]:
if_freq = 50e6

calibrations = (dm.load_IQMX_calibration_database("TEST", -6))
ro_cal = calibrations.get(frozenset(dict(lo_power=0, ssb_power=-30, lo_frequency=7.047e9, 
                                      if_frequency=0, waveform_resolution=1).items()))
ro_pb = PulseBuilder(ro_cal)

calibrations = (dm.load_IQMX_calibration_database("TESTQ", -6))
q_cal = calibrations.get(frozenset(dict(lo_power=15, ssb_power=-10, lo_frequency=7.9e9+if_freq, 
                                      if_frequency=if_freq, waveform_resolution=0.1).items()))
q_pb = PulseBuilder(q_cal)

FileNotFoundError: [Errno 2] No such file or directory: 'Data\\IQMXCalibration\\TEST.pkl'

In [94]:
ro_awg = KeysightAWG("AWG3")

In [95]:
q_awg = KeysightAWG("AWG2")

In [13]:
from drivers.Keysight_DSOX2014 import *

In [86]:
dso = Keysight_DSOX2014("DSO2014")

# Spectroscopy

## Single-tone spectroscopy

In [7]:
# reload(lib2.Measurement)

In [13]:
# reload(lib2.SingleToneSpectroscopy)
from lib2.SingleToneSpectroscopy import *

In [106]:
ro_awg.output_continuous_wave(10e6, 1, offset=0)

In [110]:
# q_pb.add_sine_pulse(100).build().plot()
ro_awg.output_pulse_sequence(q_pb.add_sine_pulse(100).add_zero_pulse(100).build())

In [115]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100,5).build())
q_awg.output_pulse_sequence(q_pb.add_zero_pulse(100).build())

In [116]:
STS = SingleToneSpectroscopy("III-anticrossing", "Xmon Al BMSTU S444 2", vna = 'vna2', src = 'yok3')
vna_parameters = {"bandwidth":2000, "freq_limits":(6.945e9, 6.96e9), "nop":201, "power":-20, "averages":1}
STS.set_fixed_parameters(vna_parameters)
currents = linspace(-0.7e-3, 1e-3, 100)
STS._src.set_current(currents[0])
sleep(1)
STS.set_swept_parameters({'current': (STS._src.set_current, currents)})
result = STS.launch()
STS._src.set_current(0);
result.visualize();

vna2 is already initialized
yok3 is already initialized
Started at:  2017-05-22 18:10:04.088058
Time left: 0 h 0 m 0.0 s, [current: 1.00e-03]: , average cycle time: 0.17 s          

In [56]:
result.visualize();

In [44]:
result.save()

## Two-tone spectroscopy

In [77]:
# reload(lib2.Measurement)
# reload(lib2.TwoToneSpectroscopyBase)
# reload(lib2.TwoToneSpectroscopy)
from lib2.TwoToneSpectroscopy import *

In [71]:
q_pb.add_sine_pulse(100, amplitude=5).build().plot()

In [82]:
ro_awg.output_continuous_wave(50e6, 2,)

In [78]:
ro_awg.output_pulse_sequence(ro_pb.add_zero_pulse(100).add_dc_pulse(100,5).add_zero_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_zero_pulse(100).add_dc_pulse(500, 5).add_zero_pulse(100).build())

In [81]:
ro_awg._visainstrument.write(":ARM:SOUR1 IMM")

(16, <StatusCode.success: 0>)

#### Qubit spectrum

In [81]:
TTS = FluxTwoToneSpectroscopy("VI-two-tone", "Xmon Al BMSTU S444 2", vna_name="vna2", 
                              mw_src_name="mxg", current_src_name="yok3")
vna_parameters = {"bandwidth":25, "freq_limits":(7.04e9, 7.06e9), "nop":1, "power":10, "averages":1}
mw_src_parameters = {"power":-20}
mw_src_frequencies = linspace(7e9, 10e9, 50)
currents = linspace(-0.7e-3, 1e-3, 100)
TTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, currents)

vna2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7060000000.0)                    at qubit's sweet spot (-0.25 mA)
Detected frequency is 7.04724 GHz, at 0.70 mU and -2.71 degrees


In [82]:
result = TTS.launch()

Started at:  2017-05-20 20:21:58.524349
Time left: 0 h 1 m 5.15 s, parameter value: -1.918e-04, average cycle time: 0.08 s           

In [113]:
result.visualize();

In [28]:
result.save()

In [20]:
result._name = "5.5GHz-two-tone-w-loop_-0.5mA-and-coil"

In [46]:
TTS._mw_src.set_output_state("OFF")

#### Linewidth from excitation power

In [185]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100, 5).build())

In [183]:
PTS = PowerTwoToneSpectroscopy("VI-powerscan", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":0, "averages":1}
mw_src_parameters = {}
mw_src_frequencies = linspace(7.88e9, 7.92e9,200)
mw_src_powers = linspace(-20, 15, 21)
current = -0.11e-3
PTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, mw_src_powers, current)
PTS._measurement_result.set_phase_units("deg")

vna2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of -0.11 mA
Detected frequency is 7.04716 GHz, at 0.10 mU and 33.83 degrees


In [187]:
result = PTS.launch()

Started at:  2017-05-10 21:15:08.685005
Time left: 0 h 10 m 10.55 s, parameter value: -2.000e+01, average cycle time: 0.15 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 9 m 25.8 s, parameter value: -2.000e+01, average cycle time: 0.14 s           

In [ ]:
vna.set_parameters(PTS._pre_measurement_vna_parameters)

In [106]:
result.visualize()

In [103]:
result.save()

In [ ]:
mxg.set_frequency(7.898e9)
mxg.set_power(-30)
mxg.set_output_state("OFF")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
ground_state_sdata = vna.get_sdata()

mxg.set_output_state("ON")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
mixed_state_sdata = vna.get_sdata()

In [ ]:
plot(abs(ground_state_sdata))
plot(abs(mixed_state_sdata))
plot((abs(ground_state_sdata)[10:]+abs(ground_state_sdata)[:-10])/2)

#### AC Stark

In [206]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100, 5).build())

In [208]:
ASTS = AcStarkTwoToneSpectroscopy("VI-ac-stark", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":1000, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-10, "averages":100}
mw_src_parameters = {"power":-15, "frequency":-1}
mw_src_frequencies = linspace(7.85e9, 7.9e9, 100)
vna_powers = linspace(-30, 0, 31)
current = -0.11e-3
ASTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, vna_powers, current)

vna2 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    at current of -0.11 mA
Detected frequency is 7.04716 GHz, at 0.25 mU and 33.57 degrees


In [21]:
result = ASTS.launch()

Started at:  2017-04-25 22:15:13.034443
Time left: 0 h 27 m 19.46 s, parameter value: -3.000e+01, average cycle time: 0.53 s          

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s, parameter value: 0.000e+00, average cycle time: 0.16 s              

In [112]:
result.visualize()

In [12]:
result.save()

In [88]:
TTS.close_devs("mxg")

__________________________

# Time-domain measurements

### Check mixer calibration

In [44]:
q_lo = TTS._mw_src
q_lo.set_output_state("ON")
q_lo.set_frequency(7.94e9)
q_lo.set_power(15)

In [56]:
q_lo.visa_instr.close()

In [49]:
TTS._vna.set_nop(1)
TTS._vna.set_power(0)
TTS._vna.set_center(7.04720e9)
TTS._vna.sweep_continuous()

In [60]:
q_awg.output_pulse_sequence(q_pb.add_zero_pulse(20).build())
ro_awg.output_pulse_sequence(ro_pb.add_zero_pulse(20).build())

In [61]:
q_awg.output_pulse_sequence(q_pb.add_sine_pulse(20,0).build())
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(20).build())

In [14]:
print(ro_cal)

### Set qubit frequency

In [83]:
TTS._current_src.set_current(-2.6e-4)

True

In [48]:
q_freq

7899180000.0

In [6]:
q_freq = 7899210000.0

In [16]:
q_freq = q_freq+0.03e6
q_freq

7899210000.0

### Rabi 

In [19]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveRabiOscillations)
from lib2.DispersiveRabiOscillations import DispersiveRabiOscillations

In [20]:
DRO = DispersiveRabiOscillations("VI-rabi", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg', )
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
exc_frequency = q_freq
excitation_durations = linspace(0, 350, 38)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":8000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRO.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
DRO.set_swept_parameters(excitation_durations)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04716 GHz, at 0.02 mU and -7.45 degrees


In [21]:
result = DRO.launch()

Started at:  2017-05-21 22:58:34.576359
Time left: 0 h 0 m 0.0 s, [excitation_duration: 3.50e+02]: , average cycle time: 2.99 s         

In [22]:
result.visualize();

In [107]:
result.save()

In [23]:
pi_pulse_duration = result.get_pi_pulse_duration()*1e3
pi_pulse_duration

67.491965151142452

In [13]:
pi_pulse_duration = 67.949209045107793

### Decay

In [146]:
# reload(lib2.VNATimeResolvedMeasurement)
# reload(lib2.DispersiveDecay)
from lib2.DispersiveDecay import *

In [150]:
DD = DispersiveDecay("VI-decay", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg', )
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
q_frequency = q_freq
readout_delays = linspace(0, 5000,20)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":10000, "pi_pulse_duration":pi_pulse_duration}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DD.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, q_frequency, rabi_sequence_parameters)
DD.set_swept_parameters(readout_delays)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04710 GHz, at 0.03 mU and -8.41 degrees


In [151]:
dd_result = DD.launch()

Started at:  2017-05-21 22:01:42.466198
Time left: 0 h 0 m 0.0 s, [readout_delay: 5.00e+03]: , average cycle time: 3.2 s          

In [154]:
dd_result.visualize();

In [116]:
dd_result.save()

### Ramsey

In [11]:
# reload(lib2.IQPulseSequence)
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveRamsey)
from lib2.DispersiveRamsey import *

In [17]:
DR = DispersiveRamsey("VI-ramsey", "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                                                                  q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
ramsey_delays = linspace(0, 2000, 100)
exc_frequency = q_freq-5e6
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
DR.set_swept_parameters(ramsey_delays)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04716 GHz, at 0.03 mU and -10.39 degrees


In [18]:
dr_result = DR.launch()

Started at:  2017-05-21 22:53:22.446506
Time left: 0 h 4 m 41.84 s, [ramsey_delay: 8.08e+01]: , average cycle time: 2.97 s       

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 0 m 26.92 s, [ramsey_delay: 1.82e+03]: , average cycle time: 2.99 s       

In [153]:
dr_result.visualize();

In [144]:
dr_result.save()

### Echo

In [157]:
# reload(lib2.VNATimeResolvedMeasurement)
# from lib2.VNATimeResolvedMeasurement import *
# reload(lib2.DispersiveHahnEcho)
from lib2.DispersiveHahnEcho import *

In [161]:
DHN = DispersiveHahnEcho("VI-hahn-echo", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":20, "averages":2}
q_frequency = q_freq
echo_delays = linspace(10, 7000, 20)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DHN.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, q_frequency, sequence_parameters)
DHN.set_swept_parameters(echo_delays)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04716 GHz, at 0.02 mU and -7.93 degrees


In [162]:
dhn_result = DHN.launch()

Started at:  2017-05-21 22:06:57.710228
Time left: 0 h 0 m 0.0 s, [echo_delay: 7.00e+03]: , average cycle time: 5.01 s         

In [148]:
dhn_result.visualize();

In [151]:
dhn_result.save()

In [32]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-hahn-echo").visualize()

(<matplotlib.figure.Figure at 0xd808da0>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000B1F7320>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x0000000011049AC8>], dtype=object),
 (None, None))

### Ramsey fringes

In [100]:
# reload(lib2.Measurement)
# reload(drivers.KeysightAWG)
reload(lib2.VNATimeResolvedDispersiveMeasurement)
reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
reload(lib2.DispersiveRamseyFringes)
from lib2.DispersiveRamseyFringes import *

In [27]:
DRF = DispersiveRamseyFringes("VI-ramsey-fringes", "Xmon Al BMSTU S444 2",
                                      vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "averages":2}
q_frequency = 7.89939e9
excitation_freqs = linspace(q_frequency-10e6, q_frequency+10e6, 200)
ramsey_delays = linspace(10, 5000, 200)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRF.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, sequence_parameters)
DRF.set_swept_parameters(ramsey_delays, excitation_freqs)

NameError: name 'DispersiveRamseyFringes' is not defined

In [28]:
drf_result = DRF.launch()

NameError: name 'DRF' is not defined

In [67]:
drf_result.visualize()

(<matplotlib.figure.Figure at 0x48ba0e48>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x0000000050AFB860>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000001A82A550>], dtype=object),
  <matplotlib.axes._axes.Axes at 0x50bf0390>])

In [68]:
drf_result.save()

### Rabi shevrons

In [139]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement2D)
# reload(lib2.DispersiveRabiChevrons)
from lib2.DispersiveRabiChevrons import *

In [104]:
DRC = DispersiveRabiChevrons("VI-rabi-shevrons", "Xmon Al BMSTU S444 2",
                                      vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "averages":2}
q_frequency = 7.89939e9
excitation_freqs = linspace(q_frequency-10e6, q_frequency+10e6, 200)
ramsey_delays = linspace(10, 5000, 200)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRC.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, sequence_parameters)
DRC.set_swept_parameters(ramsey_delays, excitation_freqs)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04710 GHz, at 0.03 mU and 8.89 degrees


In [105]:
drc_result = DRC.launch()

Started at:  2017-05-13 15:27:30.841761
Time left: 0 h 0 m 0.0 s, [excitation_duration: 5.00e+03, excitation_frequency: 7.91e+09]: , average cycle time: 0.24 s           

In [106]:
drc_result.visualize();

In [108]:
drc_result.save()

### Dispersive shift measurement

In [75]:
reload(lib2.VNATimeResolvedDispersiveMeasurement)
reload(lib2.VNATimeResolvedDispersiveMeasurement2D)
reload(lib2.TimeResolvedDispersiveShiftSpectroscopy)
from lib2.TimeResolvedDispersiveShiftSpectroscopy import *

In [94]:
DSS = TimeResolvedDispersiveShiftSpectroscopy("VI-rabi-dispersive-shift-exact-q-freq", "Xmon Al BMSTU S444 2",
                                      vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":100, "averages":2}
exc_frequency = 7.89919e9
excitation_durations = linspace(1, 3000, 100)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DSS.set_fixed_parameters(vna_parameters, q_frequency, ro_awg_params, q_awg_params, sequence_parameters)
DSS.set_swept_parameters(excitation_durations)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized


In [95]:
dss_result = DSS.launch()

Started at:  2017-05-14 22:48:05.121530
Time left: 0 h 0 m 0.0 s, [excitation_duration: 3.00e+03]: , average cycle time: 19.14 s          

In [97]:
dss_result.save()

In [76]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-rabi-dispersive-shift").visualize();

In [96]:
dss_result.visualize();

(<matplotlib.figure.Figure at 0x9f0f0b8>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000D2A8208>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000000F1CE160>], dtype=object),
  <matplotlib.axes._axes.Axes at 0xe73a6a0>])

______________________________________________

# Pulse optimization

## Amplified phase error

In [9]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveAPE)
from lib2.DispersiveAPE import *

### Rectangular pulses

In [36]:
for i in range(0, 11, 1):
    print(i)
    DAPE = DispersiveAPE("VI-APE-%d-pseudo-I-pulses"%i, "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                                q_awg_name="awg2", q_lo_name='mxg')
    vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10**(1.7*i/10), "averages":2}
    ramsey_angles = linspace(-4*pi, 4*pi, 100)
    exc_frequency = q_freq
    ro_awg_params =  {"calibration":ro_cal}
    q_awg_params = {"calibration":q_cal}
    APE_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000}
    APE_sequence_parameters.update({"half_pi_pulse_duration":pi_pulse_duration/2, 
                                    "pseudo_I_pulses_count":i, "modulating_window":'rectangular',
                                    "excitation_amplitude":1, "padding":5})
    DAPE.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, APE_sequence_parameters)
    DAPE.set_swept_parameters(ramsey_angles)
    dape_result = DAPE.launch()
    dape_result.save()

0
vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04710 GHz, at 0.02 mU and -13.45 degrees
Started at:  2017-05-21 23:02:08.364587
Time left: 0 h 0 m 0.0 s, [ramsey_angle: 1.26e+01]: , average cycle time: 1.38 s          1
vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04716 GHz, at 0.02 mU and -9.30 degrees
Started at:  2017-05-21 23:04:33.375881
Time left: 0 h 0 m 0.0 s, [ramsey_angle: 1.26e+01]: , average cycle time: 1.38 s          2
vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided freque

In [125]:
dape_result = DAPE.launch()

Started at:  2017-05-19 20:46:07.693439
Time left: 0 h 0 m 0.0 s, [ramsey_angle: 1.26e+01]: , average cycle time: 2.96 s          

In [54]:
dape_result.visualize();

In [126]:
dape_result.save()

### Gaussian pulses

#### Rabi

In [37]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersivePiPulseAmplitudeCalibration)
from lib2.DispersivePiPulseAmplitudeCalibration import *

In [40]:
DPPAC = DispersivePiPulseAmplitudeCalibration("VI-pi-amp-cal", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
exc_frequency = q_freq
excitation_amps = linspace(0, 3, 20)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000,
                            "repetition_period":8000, "excitation_duration":50, "modulating_window":"gaussian"}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DPPAC.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, sequence_parameters)
DPPAC.set_swept_parameters(excitation_amps)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04704 GHz, at 0.03 mU and 3.02 degrees


In [41]:
dppac_result = DPPAC.launch()

Started at:  2017-05-22 00:20:07.788235
Time left: 0 h 0 m 0.0 s, [excitation_amplitude: 3.00e+00]: , average cycle time: 2.96 s         

In [42]:
dppac_result.visualize();

In [7]:
pi_pulse_amplitude = 2.28
pi_pulse_amplitude

2.28

#### Ramsey 

In [110]:
from lib2.DispersiveRamsey import *

In [44]:
DR = DispersiveRamsey("VI-ramsey", "Xmon Al BMSTU S444 2", vna_name="vna2", 
                                          ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
ramsey_delays = linspace(0, 2000, 100)
exc_frequency = q_freq-5e6
ramsey_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                              "excitation_amplitude":pi_pulse_amplitude/2, "modulating_window":"gaussian",
                              "repetition_period":8000, "half_pi_pulse_duration":50}
DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, ramsey_sequence_parameters)
DR.set_swept_parameters(ramsey_delays)

vna2 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7040000000.0, 7052500000.0)                    
Detected frequency is 7.04710 GHz, at 0.03 mU and 4.10 degrees


In [45]:
drg_result = DR.launch()

Started at:  2017-05-22 00:21:44.582771
Time left: 0 h 4 m 44.01 s, [ramsey_delay: 8.08e+01]: , average cycle time: 2.99 s       

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Time left: 0 h 2 m 11.51 s, [ramsey_delay: 1.11e+03]: , average cycle time: 2.99 s       

Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x000000000F666378>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\weakref.py", line 109, in remove
    self = selfref()
KeyboardInterrupt


Time left: 0 h 2 m 8.5 s, [ramsey_delay: 1.13e+03]: , average cycle time: 2.99 s       

In [132]:
drg_result.visualize();

#### APE

In [10]:
for i in range(0, 11, 1):
    print(i)
    DAPE = DispersiveAPE("VI-APE-gaussian-%d-pseudo-I-pulses"%i, "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                                q_awg_name="awg2", q_lo_name='mxg')
    vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10**(1.7*i/10), "averages":3}
    ramsey_angles = linspace(-4*pi, 4*pi, 100)
    exc_frequency = q_freq
    ro_awg_params =  {"calibration":ro_cal}
    q_awg_params = {"calibration":q_cal}
    APE_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000, "padding":5,
                            "half_pi_pulse_duration":50, "modulating_window":"gaussian",
                            "excitation_amplitude":pi_pulse_amplitude/2, "pseudo_I_pulses_count":i}
    DAPE.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, APE_sequence_parameters)
    DAPE.set_swept_parameters(ramsey_angles)
    dape_result = DAPE.launch()
    dape_result.save()
    clear_output()

In [98]:
dape_result.visualize()

(<matplotlib.figure.Figure at 0xbbba160>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x000000000DF7E630>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000000DF34898>], dtype=object),
 (None, None))

---------------------------

In [68]:
reload(lib2.IQPulseSequence)
from lib2.IQPulseSequence import *

In [35]:
pb = PulseBuilder(q_cal)

In [53]:
APE_sequence_parameters["ramsey_angle"]=0
APE_sequence_parameters["pseudo_I_pulses_count"] = 1
APE_sequence_parameters["excitation_amplitude"] = 1

In [54]:
APE_sequence_parameters

{'awg_trigger_reaction_delay': 400,
 'excitation_amplitude': 1,
 'half_pi_pulse_duration': 50,
 'modulating_window': 'gaussian',
 'padding': 15,
 'pseudo_I_pulses_count': 1,
 'ramsey_angle': 0,
 'readout_duration': 1000,
 'repetition_period': 8000}

In [58]:
PulseBuilder.build_dispersive_APE_sequences(pb, ro_pb, APE_sequence_parameters)[0].plot()
pb.add_zero_pulse(400).add_sine_pulse(65).add_sine_pulse(50).add_zero_pulse(15).build().plot()
plt.xlim(300,600)

(300, 600)

In [63]:
# pb.add_zero_pulse(10).add_sine_pulse(30).add_zero_pulse(10).build().plot()
pb.add_zero_pulse(10).add_sine_pulse(50, window=("gaussian", {"amplitude":10})).add_zero_pulse(10).build().plot()
plt.grid()

In [56]:
rect = pb.add_zero_pulse(100).add_sine_pulse(300, amplitude=1).add_zero_pulse(1000).build()

In [57]:
gaus = pb.add_zero_pulse(100).add_sine_pulse(300, amplitude=1, window="gaussian").add_zero_pulse(1000).build()

In [58]:
rect.plot()
gaus.plot()

In [66]:
q_awg.output_pulse_sequence(gaus)

In [195]:
exp(-1/8)

0.88249690258459546

In [196]:
X = linspace(-1,1)

In [202]:
plot(X, exp(-(3*X)**2/2**2))